In [ ]:
%load_ext autoreload
%autoreload 2

# ENLEVER LES POI?TS GEOGRAPHIQUES DUPLIQUES

In [ ]:
from pathlib import Path
import gstlearn as gl
import gstlearn.plot as gp
from bramm_data_analysis import loaders

# Data

### Loading

In [ ]:
x1 = "longitude"
x2 = "latitude"
z1 = "copper"
z2 = "aluminium"

# Moss
moss_data_path = Path("../data/Mines_2024.xlsx")
observations = loaders.from_moss_csv(moss_data_path).retrieve_db(
    xs=["longitude", "latitude"], zs=z1, duplicates_handling_strategy="mean"
)

In [ ]:
extrems = observations.getExtremas()

In [ ]:
extrems

In [ ]:
step = 0.1
db_grid = gl.DbGrid.create(
    x0=[extrems[0, 0], extrems[1, 0]],
    dx=[step, step],
    nx=[
        int((extrems[0, 1] - extrems[0, 0]) / step),
        int((extrems[1, 1] - extrems[1, 0]) / step),
    ],
)

In [ ]:
varioParamMulti = gl.VarioParam.createMultiple(
    ndir=2, npas=10, dpas=0.5, toldis=0.2
)
vario2dir = gl.Vario(varioParamMulti)
err = vario2dir.compute(observations)
ax = gp.varmod(vario2dir)

In [ ]:
fitmod = gl.Model()
types = [gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.GAUSSIAN]
err = fitmod.fit(vario2dir, types=types)
ax = gp.varmod(vario2dir, fitmod)

In [ ]:
observations.setLocator(z1, gl.ELoc.Z)
observations

In [ ]:
uniqueNeigh = gl.NeighUnique.create()

In [ ]:
err = gl.kriging(
    dbin=observations,
    dbout=db_grid,
    model=fitmod,
    neigh=uniqueNeigh,
    flag_est=True,
    flag_std=True,
    flag_varz=False,
    namconv=gl.NamingConvention("SK"),
)

In [ ]:
db_grid["SK.copper.estim"]

In [ ]:
db_grid.plot("SK.copper.estim")